In [1]:
import sys
from pathlib import Path

module_path = Path("/Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2")
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

import importlib
import bcs_tms_core
# importlib.reload(bcs_tms_core)

In [2]:
from bcs_tms_preprocessing import run_stage_1
from bcs_tms_conditions import run_stage_2
from bcs_tms_soc_sessions import run_stage_3
from bcs_tms_energy import run_stage_4
from bcs_tms_soc_dynamics import run_stage_5
from bcs_tms_hotspots import run_stage_6,export_hotspot_analysis_to_pdf

In [3]:
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path.cwd()

feather_path = base_dir / "df_with_state_30days.feather"
parquet_path = base_dir / "soc_accuracy_sessions_30days.parquet"


feather_path = str(base_dir / "df_with_state_30days.feather")
parquet_path = str(base_dir / "soc_accuracy_sessions_30days.parquet")

# --- Stage 1 ---
feather_path = run_stage_1(
    date_str="2025-10-01",
    days=31,
    csv_path=None,          # default: ../oct25_can_parsed_data.csv
    feather_path=feather_path,
    vehicle_ids=None,       # default: DEFAULT_VEHICLE_IDS
)


=== DEBUG: df_with_state COLUMNS ===
['id', 'timestamp', 'mode', 'vehiclereadycondition', 'gun_connection_status', 'batt_maxtemp', 'batt_mintemp', 'batt_temp_delta', 'batt_maxvolt', 'batt_minvolt', 'volt_delta_mv', 'batt_maxtemp_pack', 'batt_mintemp_pack', 'batt_maxvolt_cell', 'batt_minvolt_cell', 'bat_voltage', 'total_battery_current', 'bat_soc', 'soh', 'dt_sec']

=== DEBUG: MODE VALUE COUNTS ===
mode
DISCHARGING    19706050
CHARGING        3910406
Name: count, dtype: int64
Stage 1 complete → /Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2/df_with_state_30days.feather


In [4]:
# --- Stage 2 ---
run_stage_2(
    feather_path=feather_path,
    per_vehicle_pdf=None,  # or a path if you also want the old matplotlib per-vehicle PDF
    fleet_pdf="battery_condition_fleet_report_30days.pdf",
    font_family="Courier New",
)

/Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2/bcs_tms_conditions.py:149: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summed = combined.groupby(combined.index).sum()


In [5]:
# --- Stage 3 ---
# bcs_tms_soc_sessions.py
run_stage_3(
    feather_path=feather_path,
    parquet_out=parquet_path,
    pdf_out="vehicle_wise_soc_accuracy_30days_clean.pdf",
    font_family="Courier New",
)

/Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2/bcs_tms_soc_sessions.py:182: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
/Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2/bcs_tms_soc_sessions.py:205: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


📄 Final SoC Accuracy PDF created → vehicle_wise_soc_accuracy_30days_clean.pdf
Stage 3 complete → /Users/apple/Documents/naarni/repo/explore_data/4.BCS_TMS_analysis/ver2/soc_accuracy_sessions_30days.parquet, vehicle_wise_soc_accuracy_30days_clean.pdf


In [6]:
# --- Stage 4 ---
# bcs_tms_energy.py
run_stage_4(
    parquet_path=parquet_path,
    out_csv="weighted_energy_summary_30days.csv",
)

Stage 4 complete → weighted_energy_summary_30days.csv


In [7]:
# --- Stage 5 ---
# bcs_tms_soc_dynamics.py
run_stage_5(
    feather_path=feather_path,
    pdf_path="soc_dynamics_and_c_rate_report_30days.pdf",
    font_family="Courier New",
)

📄 SoC Occupancy PDF created → soc_occupancy_report.pdf
📄 SoC Jump/Drop PDF created → soc_jumps_report.pdf
📄 C-Rate PDF created → c_rate_report.pdf
Stage 5 complete — all three modular reports generated.


In [8]:
# --- Stage 6 ---
(
    fleet_pack_hotspots,   # NEW
    fleet_temp_hotspots,
    veh_temp_hotspots,
    fleet_voltage_hotspots,
    veh_voltage_hotspots
) = run_stage_6(feather_path)


export_hotspot_analysis_to_pdf(
    fleet_pack_hotspots,
    fleet_temp_hotspots,
    veh_temp_hotspots,
    fleet_voltage_hotspots,
    veh_voltage_hotspots,
    output_pdf="hotspot_analysis_30days.pdf",
    font_family="Courier New",
)


Stage 6 complete (hotspot tables ready).
📄 Hotspot Analysis PDF saved → hotspot_analysis_30days.pdf


In [9]:
fleet_temp_hotspots

,tc_id,pack_id,c_max_cnt,c_max_pct,dc_max_cnt,dc_max_pct,c_min_cnt,c_min_pct,dc_min_cnt,dc_min_pct
0,1,1,5,0.00,306,0.00,39764,1.32,122256,0.79
1,2,1,3079,0.10,3901,0.03,17374,0.58,132878,0.86
2,3,1,1408,0.05,10328,0.07,335,0.01,11690,0.08
3,4,1,4741,0.16,70458,0.45,74,0.00,300,0.00
4,5,1,8144,0.27,140093,0.90,26450,0.88,253463,1.63
...,...,...,...,...,...,...,...,...,...,...
103,104,12,42956,1.42,198583,1.28,0,0.00,0,0.00
104,105,12,1553,0.05,6985,0.04,0,0.00,0,0.00
105,106,12,13126,0.43,182958,1.18,0,0.00,0,0.00
106,107,12,67216,2.23,375332,2.42,0,0.00,0,0.00
